In [2]:
from collections import namedtuple
import numpy as np
import math

In [3]:
Graph = namedtuple("Graph", ["nodes", "edges"])

nodes = ["A","B","C","D"]
edges = [
    ("A","B"),
    ("A","B"),
    ("A","C"),
    ("A","C"),
    ("A","D"),
    ("B","D"),
    ("C","D")
]

G = Graph(nodes,edges) 

adjacency_dict = {node:[] for node in G.nodes}

for edge in G.edges:
    node1, node2 = edge[0], edge[1]
    adjacency_dict[node1].append(node2)
    adjacency_dict[node2].append(node1)

adjacency_dict

{'A': ['B', 'B', 'C', 'C', 'D'],
 'B': ['A', 'A', 'D'],
 'C': ['A', 'A', 'D'],
 'D': ['A', 'B', 'C']}

In [4]:
[[5 for i in range(3)] for j in range(5)]

[[5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5]]

In [5]:
nodes = range(4)
edges = [
    (0,1),
    (0,1),
    (0,2),
    (0,2),
    (0,3),
    (1,3),
    (2,3)
]

G = Graph(nodes,edges)

adj = [[0 for node in G.nodes] for node in G.nodes] 

for edge in G.edges:
    node1,node2 = edge[0], edge[1]
    adj[node1][node2] += 1
    adj[node2][node1] += 1
adj

[[0, 2, 2, 1], [2, 0, 0, 1], [2, 0, 0, 1], [1, 1, 1, 0]]

<h1>Generating an adjacency matrix for a graph</h1>



<p>We define the graph nodes and edges as follows:</p>
<img src="graphs.png" height="400" >
<p> Then we generate a graph object using namedtuple from the collections module. </p>
<p>We then define the edges manually to then generate an adjacency matrix</p>

<h3>Define the graph</h3>

In [6]:
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,1),
    (0,5),
    (0,6),
    (1,0),
    (1,2),
    (1,6),
    (2,1),
    (2,3),
    (2,6),
    (3,2),
    (3,4),
    (3,6),
    (4,3),
    (4,5),
    (4,6),
    (5,4),
    (5,6),
    (5,0),
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [7]:
numberofNodes = len(G.nodes)
matrix_shape = (numberofNodes, numberofNodes)
adj_matrix = np.zeros(matrix_shape)
for edge in G.edges:
    node1, node2 = edge[0], edge[1]
    adj_matrix[node1][node2] +=1
    adj_matrix[node2][node1] +=1

adj_matrix = adj_matrix/2 #just to make everything 1s and 0s



<h1>Creating transition matrices</h1>

      
<p>The following adjacency matrix was generated by the above code for the graph(see below):</p>
<img src="graphs.png" height="400" >
      
         0   1   2   3   4   5   6
      0 [0., 1., 0., 0., 0., 1., 1.]
      1 [1., 0., 1., 0., 0., 0., 1.]
      2 [0., 1., 0., 1., 0., 0., 1.]
      3 [0., 0., 1., 0., 1., 0., 1.]
      4 [0., 0., 0., 1., 0., 1., 1.]
      5 [1., 0., 0., 0., 1., 0., 1.]
      6 [1., 1., 1., 1., 1., 1., 0.]

<p>Using this adjacency matrix, we can easily create the transition matrix based off the probabilities defined in the figure.</p>
<p>The stay probability of a node is defined by the looping arrow.</p>
   <p>This stay probability is simply the edge connecting the node to itself. This is represented by the <strong>diagonal</strong> of the adjacency matrix.</p>
<p>The transition probability of a node (to the next node) is simply then : (1 - stay probability of that node) / number of adjacent nodes.</p>

In [8]:
def generate_transitionMatrix(graph,stay_probability=0):
    """Function to create the transition matrix. We assume that the graph has already been created and the 
    relevant nodes and edges are clearly defined. """

    #get the number of nodes
    numberofNodes = len(graph.nodes)
    #get shape of the transition matrix 
    matrix_shape = (numberofNodes, numberofNodes)
    #create the adjacent matrix with zeros
    adj_matrix = np.zeros(matrix_shape)

    for edge in graph.edges:
        node1,node2 = edge[0],edge[1]
        adj_matrix[node1][node2] += 1
        adj_matrix[node2][node1] += 1
    adj_matrix = adj_matrix/2 #just to make everything 1s and 0s
    
    #lets keep the calculation of the transition probabilites dynamic based off the number of adjacent nodes from
    #a starting node and the stay probability of that starting node
    #therefore what we should have is: transition probability = (1 - stay probability) / number of adjacent nodes

    np.fill_diagonal(adj_matrix,stay_probability) #set the stay probability in the adj matrix for the for loop 
    transition_matrix = np.copy(adj_matrix)

    #lets get the number of neighbours for each node by reading off the total number of 1s (adjacent nodes)
    for index,row in enumerate(adj_matrix):
        #we assume that the loops of a node to itself are already defined in the edges of the graph
        number_adjacent_nodes = np.count_nonzero(row == 1)
        transition_probability = (1 - stay_probability) / (number_adjacent_nodes)

        #adjust the decimal places
        transition_probability

        #replace the 1s (adjacent nodes) with their transition probability
        transition_matrix[index] = np.where(row ==1, f"{transition_probability:.2f}", row) #formatting the transition probability to two decimal places

    #set the stay probability of the matrix for each node 
    np.fill_diagonal(transition_matrix,stay_probability)
    #replace the diagonals with zeros for the adjacent matrix
    np.fill_diagonal(adj_matrix, 0)

    return adj_matrix, transition_matrix
        



In [9]:
#define graph, nodes and edges
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,0), #loop back to itself
    (0,1),
    (0,5),
    (0,6),
    (1,1), #loop back to itself
    (1,0),
    (1,2),
    (1,6),
    (2,2), #loop back to itself
    (2,1),
    (2,3),
    (2,6),
    (3,3), #loop back to itself
    (3,2),
    (3,4),
    (3,6),
    (4,4), #loop back to itself
    (4,3),
    (4,5),
    (4,6),
    (5,5), #loop back to itself
    (5,4),
    (5,6),
    (5,0),
    (6,6), #loop back to itself
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [10]:
colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G, stay_probability=0.1)
colour_adjmatrix, colour_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.1 , 0.3 , 0.  , 0.  , 0.  , 0.3 , 0.3 ],
        [0.3 , 0.1 , 0.3 , 0.  , 0.  , 0.  , 0.3 ],
        [0.  , 0.3 , 0.1 , 0.3 , 0.  , 0.  , 0.3 ],
        [0.  , 0.  , 0.3 , 0.1 , 0.3 , 0.  , 0.3 ],
        [0.  , 0.  , 0.  , 0.3 , 0.1 , 0.3 , 0.3 ],
        [0.3 , 0.  , 0.  , 0.  , 0.3 , 0.1 , 0.3 ],
        [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.1 ]]))

In [11]:
texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G, stay_probability=0.3)
texture_adjmatrix, texture_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.3 , 0.23, 0.  , 0.  , 0.  , 0.23, 0.23],
        [0.23, 0.3 , 0.23, 0.  , 0.  , 0.  , 0.23],
        [0.  , 0.23, 0.3 , 0.23, 0.  , 0.  , 0.23],
        [0.  , 0.  , 0.23, 0.3 , 0.23, 0.  , 0.23],
        [0.  , 0.  , 0.  , 0.23, 0.3 , 0.23, 0.23],
        [0.23, 0.  , 0.  , 0.  , 0.23, 0.3 , 0.23],
        [0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.3 ]]))

In [12]:
shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G, stay_probability=0.5)
shape_adjmatrix, shape_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.5 , 0.17, 0.  , 0.  , 0.  , 0.17, 0.17],
        [0.17, 0.5 , 0.17, 0.  , 0.  , 0.  , 0.17],
        [0.  , 0.17, 0.5 , 0.17, 0.  , 0.  , 0.17],
        [0.  , 0.  , 0.17, 0.5 , 0.17, 0.  , 0.17],
        [0.  , 0.  , 0.  , 0.17, 0.5 , 0.17, 0.17],
        [0.17, 0.  , 0.  , 0.  , 0.17, 0.5 , 0.17],
        [0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.5 ]]))

<h1>Creating Design Files - Pseudo Code</h1>
<p>Create an empty dataframe which should hold columns for</p>
    <ul>
    <li>Participant ID</li>
    <li>Block Number</li>
    <li>Trial Number</li>
    <li>Colour_stim</li>
    <li>Texture_stim</li>
    <li>Shape_stim</li>
    <li>Colour_choice</li>
    <li>Texture_choice</li>
    <li>Shape_stim</li>
    </ul>
<p>Set a variable for number of trials</p>
<p>Set a variable for number of participants for which we need to create design files</p>
<p>Set a flag for every 20th trial to trigger a choice trial.</p>
<p>Set up an array containing the three stay probabilities [0.1, 0.3, 0.5]. The stay probabilities need to be included as a parameter into the function to generate transition probabilities </p>
<p>Loop through the range of participants</p>
    <p>Pick a transition probability randomly for each of the dimensions. We do this to ensure participants  experience fast,medium and slow transitions of the dimensions </p>
    <p>Set up adjacency and transition matrices for each dimension using the functions with the chosen transition probs in the step above.</p>
    <p>Set up variables to hold current rows for colour, texture and shape</p>
    <p>Set up a variable for block, update this every 180 trials by +1. </p>
    <p>Loop through number of trials : </p>
        <ul>
        <li>Just for the first iteration, generate 3 random whole numbers between 1-7. These 3 numbers will be the index of the row of the transition matrix of each dimension. Record these 3 numbers as the current row</li>
        <li>These resulting numbers will be used to generate the first object (colour + texture + shape) </li>
        <li>From the second iteration onwards (for all the following shapes), generate 3 random numbers between 0 - 1.</li>
            <li>If the randomly generated number for all or any dimension, is greater than the stay probability of that dimension, we transition to the adjacent node. To transition, we do the following:</li>
            <li>To determine which node (so basically decide the next row) to transition to based off the current row, we look for adjacent nodes (columns) in the current row of the matrix where != 0.</li>
            <li>Note that sometimes the stay probabilities are greater than the transition probabilities.</li>
            <li> If the current row != 6 (7th row), then we follow the steps below to find the node to transition-</li>
            <li>If difference <= 0.4, transition to first neighbour, get column index, this number/index becomes current row. Update the respective dimension column(s) for that trial with the row number</li>
            <li>If difference >0.4 & <= 0.7, transition to second neighbour, get column index, this number/index becomes current row.</li>
            <li>If difference >0.7 , transition to third neighbour, get column index, this number/index becomes current row.</li>
            <li>If row == 6, then we randomly pick a neighbour to transition to.</li>
            <li>Every 20th trial, or rather If the trial%20 == 0, then we generate 3 random whole numbers for the choice columns for the choice trials  </li>
            <li>If the randomly generated number for all or any dimension, is less than stay probability, we don't transition. Therefore current row remains the same</li>
            <li>To include our choice trials, if trial % 20, we draw also draw 3 random numbers and add it to the colour_choice, texture_choice and shape_choice rows of the dataframe. Otherwise they remain NaNs</li>
            </ul>

In [13]:
import pandas as pd
import numpy as np
import random

In [14]:
#define graph, nodes and edges
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,0), #loop back to itself
    (0,1),
    (0,5),
    (0,6),
    (1,1), #loop back to itself
    (1,0),
    (1,2),
    (1,6),
    (2,2), #loop back to itself
    (2,1),
    (2,3),
    (2,6),
    (3,3), #loop back to itself
    (3,2),
    (3,4),
    (3,6),
    (4,4), #loop back to itself
    (4,3),
    (4,5),
    (4,6),
    (5,5), #loop back to itself
    (5,4),
    (5,6),
    (5,0),
    (6,6), #loop back to itself
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [15]:
def generateChoiceTrials(total_trials, n_choice_trials, start_trial, jitter_range):
    """Function generates an array containing choice trials for the participants. The start trial parameter is used to
    to define the first choice trial. The total trials are of the experiment. The parameter
    n_choicetrials is the number of choice trials the participant gets. We define the starting trial and a jitter range.
    The minimum interval between trials are calculated inside the function and a random jitter from 0 to 5 is added
    to the interval. This makes the trials less predictable"""

    #generate the random choice trials between 1 and 240 trials with the first choice trial appearing at trial 10 or later

    #lets start with generating random trial numbers which are greater than 10 but less than 20
    #we start with a first trial being a number between 10 and 15
    # subsequent trials are generated if trial is greater than the previous trial and if the trial is between 10 and 20 increments from the previous trial
    #these trials must be a total of 18 trials and should be spaced out so they fill the span of 240 total trials
    minimum_interval = (total_trials - start_trial) // n_choice_trials
    choice_trials = []

    for i in range(n_choice_trials):
        base = start_trial + i * minimum_interval
        jitter = random.randint(0, jitter_range)
        subsequent_trial = min(base + jitter, total_trials - 1)
        choice_trials.append(subsequent_trial)

    return choice_trials

a = generateChoiceTrials(240*8,18*8,10,5)
a

[14,
 27,
 41,
 52,
 64,
 78,
 89,
 106,
 114,
 130,
 141,
 156,
 166,
 179,
 193,
 207,
 220,
 234,
 249,
 261,
 271,
 288,
 301,
 313,
 327,
 340,
 352,
 364,
 374,
 388,
 403,
 417,
 430,
 443,
 452,
 467,
 480,
 491,
 507,
 522,
 534,
 544,
 560,
 569,
 582,
 596,
 613,
 622,
 638,
 649,
 662,
 677,
 690,
 704,
 713,
 727,
 742,
 753,
 764,
 780,
 790,
 807,
 816,
 829,
 847,
 855,
 870,
 883,
 898,
 909,
 924,
 936,
 951,
 962,
 974,
 988,
 1000,
 1013,
 1025,
 1042,
 1054,
 1064,
 1076,
 1089,
 1104,
 1116,
 1131,
 1143,
 1155,
 1172,
 1182,
 1193,
 1209,
 1219,
 1233,
 1246,
 1258,
 1272,
 1285,
 1299,
 1310,
 1324,
 1340,
 1351,
 1364,
 1380,
 1390,
 1401,
 1419,
 1429,
 1443,
 1453,
 1471,
 1481,
 1497,
 1505,
 1522,
 1532,
 1544,
 1560,
 1572,
 1587,
 1599,
 1611,
 1622,
 1640,
 1652,
 1663,
 1676,
 1691,
 1703,
 1714,
 1727,
 1739,
 1753,
 1769,
 1780,
 1794,
 1806,
 1822,
 1831,
 1848,
 1861,
 1869]

In [16]:
   #could introduce a while loop here to ensure at least one of the dimensions of the randomly generated
                #numbers are not consistent with the graph structure for the choice trial

In [17]:
stayProbs = [0.1,0.3,0.5]
random.shuffle(stayProbs)
slow = np.argmax(stayProbs)
medium = np.where(np.array(stayProbs) ==0.3)[0][0]
fast = np.argmin(stayProbs)

dimensionSpeedMap = {
    0 : "Colour",
    1 : "Shape",
    2 : "Texture"
}

dimensionSpeeds = {
    "slow": dimensionSpeedMap[slow] ,
    "medium": dimensionSpeedMap[medium],
    "fast": dimensionSpeedMap[fast]
}

print(stayProbs)


print(dimensionSpeeds)



[0.3, 0.1, 0.5]
{'slow': 'Texture', 'medium': 'Colour', 'fast': 'Shape'}


In [55]:
master_df = []

nTrials = 200
nParticipants = 1
nBlocks = 8
nSessions = 2
stayProbs = [0.1,0.3,0.5]
block = 1 #starting with just the first block



for participant in range(nParticipants):
    
    #create dataframe for design file for every participant
    #regular columns
    columns = ["Participant_ID",
    "Block Number",
    "Session Number",
    "Trial Number",
    "Stay_Probability(Colour)",
    "Stay_Probability(Texture)",
    "Stay_Probability(Shape)",
    "Slow_stim", #shows the node of the slow dimension (e.g.: if colour is the slower changing dimension, the node will be shown here. This is the same as the value in "Colour_stim")
    "Medium_stim",
    "Fast_stim",
    "Colour_stim",
    "Texture_stim",
    "Shape_stim",
    "Choice Trial Index",
    "Colour_distractor",
    "Texture_distractor",
    "Shape_distractor",
    "Slow_distractor", #shows the node of the distractor in terms of the slow dimension (e.g.: if colour is the slower changing dimension for the stimulus, the node for colour of the distractor will be shown here. This is the same as the value in "Colour_distractor")
    "Medium_distractor",
    "Fast_distractor",
    "Distractor_consistent_colour",
    "Distractor_consistent_texture",
    "Distractor_consistent_shape",
    "Distractor_consistent_slow",
    "Distractor_consistent_medium",
    "Distractor_consistent_fast",
    "Distractor_shares_colour", #column with 1s and 0s to show if the randomly generated distractor has the same dimensions as the stimulus
    "Distractor_shares_texture",
    "Distractor_shares_shape",
    "Distractor_shares_slow", #column with 1s and 0s to show if the randomly generated distractor shows the same dimensions as the stimulus, however just mapped based off the speed 
    "Distractor_shares_medium",
    "Distractor_shares_fast"
    ]


    #prepare the stayprobabilities and transition matrices for each dimension
    random.shuffle(stayProbs)
    #assign the respective dimensions to the speeds in which they transition
    slow = np.argmax(stayProbs)
    medium = np.where(np.array(stayProbs) ==0.3)[0][0]
    fast = np.argmin(stayProbs)

    # the speed map is to just map the nodes on to the dimension, based off the speed in which it changes
    # e.g. if colour has a stay prob of 0.1, this dimension changes faster, so a node of 2 in the Colour_stim column
    # would be mapped to the Stim_fast column
    dimensionSpeedMap = {
        0 : "Colour",
        1 : "Texture",
        2 : "Shape"
    }

    dimensionSpeeds = {
        "Slow": dimensionSpeedMap[slow] ,
        "Medium": dimensionSpeedMap[medium],
        "Fast": dimensionSpeedMap[fast]
    }

    colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[0])
    texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[1])
    shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[2])

    # this map is to check if the randomly generated nodes for a distractor in a choice trial are consistent with
    # the preceeding stimulus object in the graph structure. If the preceeding stimulus object has the nodes 1 1 1 (Colour,Texture,Shape)
    # and the distractor has the nodes 2 2 4 (Colour, Texture, Shape), then its nodes are consistent with the graph structure
    # as these nodes are neighbours. Therefore we populate the distractor_consistent_colour, distractor_consistent_texture with 1s and 0 for distractor_consistent_shape
    possibleNeighbours = {
        0 : [0,1,5,6],
        1 : [1,0,2,6],
        2 : [2,1,3,6],
        3 : [3,2,4,6],
        4 : [4,3,5,6],
        5 : [5,4,0,6],
        6 : [0,1,2,3,4,5,6]
    }


    for session in range(nSessions):
        df = pd.DataFrame(columns=columns)
        rowIndex = 0
        for block in range(nBlocks):
            choiceTrialFlag = generateChoiceTrials(nTrials,18,10,5)

            
            currentRow_colour = None
            currentRow_shape = None
            currentRow_texture = None
            choiceTrialIndex = 1
            
            for trial in range(nTrials):
                df.loc[rowIndex,"Block Number"] = block + 1
                df.loc[rowIndex, "Session Number"] = session + 1
                df.loc[rowIndex,"Trial Number"] = trial + 1
                df.loc[rowIndex,"Stay_Probability(Colour)"] = stayProbs[0]
                df.loc[rowIndex,"Stay_Probability(Texture)"] = stayProbs[1]
                df.loc[rowIndex,"Stay_Probability(Shape)"] = stayProbs[2]

                stimNodeDimensionSpeedMap = {
                    "Colour": None,
                    "Texture": None,
                    "Shape": None
                }
            
                if trial == 0: #for the very first trial generate 3 random numbers (integers)
                    color_int = random.randint(0,6)
                    texture_int = random.randint(0,6)
                    shape_int = random.randint(0,6)

                    df.loc[rowIndex, "Colour_stim"] = color_int
                    df.loc[rowIndex, "Texture_stim"] = texture_int
                    df.loc[rowIndex, "Shape_stim"] = shape_int

                    currentRow_colour = color_int
                    currentRow_texture = texture_int
                    currentRow_shape = shape_int

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]


                else:
                    color_int = float(f"{random.uniform(0,1):.2f}")
                    texture_int = float(f"{random.uniform(0,1):.2f}")
                    shape_int = float(f"{random.uniform(0,1):.2f}")

                    ##For the Colour dimension
                    
                    if color_int > stayProbs[0]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number is greater than stay probability, we can transition")

                        colour_node = colour_transition_matrix[currentRow_colour] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((colour_node != 0) & (colour_node != stayProbs[0]))
                        adj_node_transprobs = [colour_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {colour_node}, Stay probability : {stayProbs[0]}, Random Number Drawn:{color_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")
                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_colour == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour
                        
                        else:
                            if color_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            elif color_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                        

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number: {color_int} is lower than stay probability {stayProbs[0]}, we stay", "\n")
                        df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                    ##For the texture dimension

                    if texture_int > stayProbs[1]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number is greater than stay probability, we can transition")
                        texture_node = texture_transition_matrix[currentRow_texture] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((texture_node != 0) & (texture_node != stayProbs[1]))
                        adj_node_transprobs = [texture_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {texture_node}, Stay probability : {stayProbs[1]}, Random Number Drawn:{texture_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_texture == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture
                        
                        else:
                            if texture_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")

                            elif texture_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number: {texture_int} is lower than stay probability {stayProbs[1]}, we stay", "\n")
                        df.loc[rowIndex, "Texture_stim"] = currentRow_texture


                    ##For the shape dimension
                    
                    if shape_int > stayProbs[2]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number is greater than stay probability, we can transition")
                        shape_node = shape_transition_matrix[currentRow_shape] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((shape_node != 0) & (shape_node != stayProbs[2]))
                        adj_node_transprobs = [shape_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {shape_node}, Stay probability : {stayProbs[2]}, Random Number Drawn:{shape_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_shape == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape
                        
                        else:
                            if shape_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {shape_node}","\n")

                            elif shape_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number: {shape_int} is lower than stay probability {stayProbs[2]}, we stay", "\n")
                        df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]

                    if trial in choiceTrialFlag :
                        color_int_distractor = random.randint(0,6)
                        texture_int_distractor = random.randint(0,6)
                        shape_int_distractor = random.randint(0,6)

                        #checking if the distractor dimensions are consistent with the preceeding stimulus object
                        #get the stimlus object nodes from preceeding trial, so df.loc[trial - 1, "Colour_stim"]
                        #get also the stimulus object from preceding trial based off speed, so df.loc[trial-1, "Stim_fast or whatever"]
                        #get the neighbour array from the possible neighbours map, if any or all of the distractor stimulus nodes are in the array, it means
                        #they are consistent and therefore the respective dimension gets marked with a 1 and inconsistent ones with a 0
                        relevantCols = {
                            "dimensions" : ["Colour_stim", "Texture_stim", "Shape_stim"],
                            "dimensions_speed": ["Slow_stim", "Medium_stim", "Fast_stim"]
                        }
                        
                        colour_preced = None
                        texture_preced = None
                        shape_preced = None
                        slow_preced = None
                        medium_preced = None
                        fast_preced = None

                        for col in relevantCols:
                            if col == "dimensions":
                                colour_preced = df.loc[rowIndex-1, relevantCols[col][0]]
                                texture_preced = df.loc[rowIndex-1, relevantCols[col][1]]
                                shape_preced = df.loc[rowIndex-1, relevantCols[col][2]]
                            else:
                                slow_preced = df.loc[rowIndex-1, relevantCols[col][0]]
                                medium_preced = df.loc[rowIndex-1, relevantCols[col][1]]
                                fast_preced = df.loc[rowIndex-1, relevantCols[col][2]]

                        nodes_preced = [colour_preced, texture_preced,shape_preced]
                        nodes_speed_preced = [slow_preced,medium_preced,fast_preced]
                        distractorSpeedVals = [
                            df.loc[rowIndex, "Slow_distractor"],
                            df.loc[rowIndex, "Medium_distractor"],
                            df.loc[rowIndex, "Fast_distractor"],
                        ]

                        consistentCols = ["Distractor_consistent_colour",
                                          "Distractor_consistent_texture",
                                          "Distractor_consistent_shape"]
                        
                        consistentSpeedCols = ["Distractor_consistent_slow",
                                               "Distractor_consistent_medium",
                                               "Distractor_consistent_fast"]
                        
                        

                        while True:
                            color_int_distractor = random.randint(0,6)
                            texture_int_distractor = random.randint(0,6)
                            shape_int_distractor = random.randint(0,6) 
                            match_count = 0



                            if color_int_distractor in possibleNeighbours[colour_preced] and colour_preced !=6 and color_int_distractor == colour_preced and color_int_distractor == 6:
                                match_count +=1
                            if shape_int_distractor in possibleNeighbours [shape_preced] and shape_preced != 6 and shape_int_distractor == shape_preced and shape_int_distractor ==6:
                                match_count +=1
                            if texture_int_distractor in possibleNeighbours[texture_preced] and texture_preced !=6 and texture_int_distractor == texture_preced and texture_int_distractor ==6:
                                match_count +=1

                            if match_count <= 2:
                                break



                        # while (
                        #     #ensure distractor shares no dimension with preceding stimulus
                        #     (color_int_distractor not in possibleNeighbours[colour_preced]) and \
                        #     (shape_int_distractor not in possibleNeighbours [shape_preced]) and \
                        #     (texture_int_distractor not in possibleNeighbours[texture_preced]) 
                            
                        #     # or (
                        #     # #check if the distractor node values are identical to the preceding stimulus object
                        #     # (color_int_distractor == colour_preced) and \
                        #     # (shape_int_distractor == shape_preced) and \
                        #     # (texture_int_distractor == texture_preced)
                        #     # )

                        #     # or
                        #     #ensures not all distractor node values are neighbours of the preceding stimulus object 
                        #     # (
                        #     # (color_int_distractor in possibleNeighbours[colour_preced]) and \
                        #     # (shape_int_distractor in possibleNeighbours [shape_preced]) and \
                        #     # (texture_int_distractor in possibleNeighbours[texture_preced])
                        #     # )
                        #     ):
                            
                        #     ##print("Distractor shares all dimensions")
                        #     color_int_distractor = random.randint(0,6)
                        #     texture_int_distractor = random.randint(0,6)
                        #     shape_int_distractor = random.randint(0,6)      

                       
                        

                        distractorNodeDimensionsMap = {
                            "Colour" : color_int_distractor,
                            "Texture" : texture_int_distractor,
                            "Shape" : shape_int_distractor
                        }
                        
                        df.loc[rowIndex, "Choice Trial Index"] = choiceTrialIndex
                        df.loc[rowIndex, "Colour_distractor"] = color_int_distractor
                        df.loc[rowIndex, "Texture_distractor"] = texture_int_distractor
                        df.loc[rowIndex, "Shape_distractor"] = shape_int_distractor
                        df.loc[rowIndex, "Slow_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Slow"]]
                        df.loc[rowIndex, "Medium_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Medium"]]
                        df.loc[rowIndex, "Fast_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]

                        stimNodeVals = [currentRow_colour,currentRow_texture,currentRow_shape]
                        distractorNodeVals = [color_int_distractor, texture_int_distractor, shape_int_distractor]
                        colsToModify = ["Distractor_shares_colour", "Distractor_shares_texture","Distractor_shares_shape"]

                        stimNodeSpeedVals = [
                            stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]],
                            stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]],
                            stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]
                        ]
                        distractorNodeSpeedVals = [
                            distractorNodeDimensionsMap[dimensionSpeeds["Slow"]],
                            distractorNodeDimensionsMap[dimensionSpeeds["Medium"]],
                            distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]
                        ]
                        speedColsToModify = ["Distractor_shares_slow", "Distractor_shares_medium", "Distractor_shares_fast"]
                    

                        for index, value in enumerate(distractorNodeVals):
                            if value == stimNodeVals[index]:
                                df.loc[rowIndex, colsToModify[index]] = 1
                            else:
                                df.loc[rowIndex,colsToModify[index]] = 0

                        for index, value in enumerate(distractorNodeSpeedVals):
                            if value == stimNodeSpeedVals[index]:
                                df.loc[rowIndex,speedColsToModify[index]] = 1
                            else:
                                df.loc[rowIndex,speedColsToModify[index]] = 0

                     
                        #if distractor dimension values are consistent with the preceding stimulus object == 1, else == 0
                        for index,val in enumerate(nodes_preced):
                            if distractorNodeVals[index] in possibleNeighbours[val]:
                                df.loc[rowIndex, consistentCols[index]] = 1
                            else:
                                df.loc[rowIndex, consistentCols[index]] = 0

                        for index,val in enumerate(nodes_speed_preced):
                            if distractorSpeedVals[index] in possibleNeighbours[val]:
                                df.loc[rowIndex, consistentSpeedCols[index]] = 1
                            else:
                                df.loc[rowIndex, consistentSpeedCols[index]] = 0


                        
                    
                        choiceTrialIndex += 1
                        

                rowIndex += 1

        

        df.iloc[:,0] = participant + 1
        #df.to_csv(f"../Experiment (No Backend)/public/DesignFiles/DesignFile_{participant+1}_session{session+1}.csv", index=False)
        df.to_csv(f"DesignFile_{participant+1}_session{session+1}.csv", index=False)
        
                    

                






C:\Users\Rishi\AppData\Local\Temp\ipykernel_17904\1408226557.py:445: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,0] = participant + 1
C:\Users\Rishi\AppData\Local\Temp\ipykernel_17904\1408226557.py:445: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,0] = participant + 1


In [56]:
master_df = []

nTrials = 200
nParticipants = 1
nBlocks = 8
nSessions = 2
stayProbs = [0.1,0.3,0.5]
block = 1 #starting with just the first block



for participant in range(nParticipants):
    
    #create dataframe for design file for every participant
    #regular columns
    columns = ["Participant_ID",
    "Block Number",
    "Session Number",
    "Trial Number",
    "Stay_Probability(Colour)",
    "Stay_Probability(Texture)",
    "Stay_Probability(Shape)",
    "Slow_stim", #shows the node of the slow dimension (e.g.: if colour is the slower changing dimension, the node will be shown here. This is the same as the value in "Colour_stim")
    "Medium_stim",
    "Fast_stim",
    "Colour_stim",
    "Texture_stim",
    "Shape_stim",
    "Choice Trial Index",
    "Colour_distractor",
    "Texture_distractor",
    "Shape_distractor",
    "Slow_distractor", #shows the node of the distractor in terms of the slow dimension (e.g.: if colour is the slower changing dimension for the stimulus, the node for colour of the distractor will be shown here. This is the same as the value in "Colour_distractor")
    "Medium_distractor",
    "Fast_distractor",
    "Distractor_consistent_colour",
    "Distractor_consistent_texture",
    "Distractor_consistent_shape",
    "Distractor_consistent_slow",
    "Distractor_consistent_medium",
    "Distractor_consistent_fast",
    "Distractor_shares_colour", #column with 1s and 0s to show if the randomly generated distractor has the same dimensions as the stimulus
    "Distractor_shares_texture",
    "Distractor_shares_shape",
    "Distractor_shares_slow", #column with 1s and 0s to show if the randomly generated distractor shows the same dimensions as the stimulus, however just mapped based off the speed 
    "Distractor_shares_medium",
    "Distractor_shares_fast"
    ]


    #prepare the stayprobabilities and transition matrices for each dimension
    random.shuffle(stayProbs)
    #assign the respective dimensions to the speeds in which they transition
    slow = np.argmax(stayProbs)
    medium = np.where(np.array(stayProbs) ==0.3)[0][0]
    fast = np.argmin(stayProbs)

    # the speed map is to just map the nodes on to the dimension, based off the speed in which it changes
    # e.g. if colour has a stay prob of 0.1, this dimension changes faster, so a node of 2 in the Colour_stim column
    # would be mapped to the Stim_fast column
    dimensionSpeedMap = {
        0 : "Colour",
        1 : "Texture",
        2 : "Shape"
    }

    dimensionSpeeds = {
        "Slow": dimensionSpeedMap[slow] ,
        "Medium": dimensionSpeedMap[medium],
        "Fast": dimensionSpeedMap[fast]
    }

    colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[0])
    texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[1])
    shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[2])

    # this map is to check if the randomly generated nodes for a distractor in a choice trial are consistent with
    # the preceeding stimulus object in the graph structure. If the preceeding stimulus object has the nodes 1 1 1 (Colour,Texture,Shape)
    # and the distractor has the nodes 2 2 4 (Colour, Texture, Shape), then its nodes are consistent with the graph structure
    # as these nodes are neighbours. Therefore we populate the distractor_consistent_colour, distractor_consistent_texture with 1s and 0 for distractor_consistent_shape
    possibleNeighbours = {
        0 : [0,1,5,6],
        1 : [1,0,2,6],
        2 : [2,1,3,6],
        3 : [3,2,4,6],
        4 : [4,3,5,6],
        5 : [5,4,0,6],
        6 : [0,1,2,3,4,5,6]
    }


    for session in range(nSessions):
        df = pd.DataFrame(columns=columns)
        rowIndex = 0
        for block in range(nBlocks):
            choiceTrialFlag = generateChoiceTrials(nTrials,18,10,5)

            
            currentRow_colour = None
            currentRow_shape = None
            currentRow_texture = None
            choiceTrialIndex = 1
            
            for trial in range(nTrials):
                df.loc[rowIndex,"Block Number"] = block + 1
                df.loc[rowIndex, "Session Number"] = session + 1
                df.loc[rowIndex,"Trial Number"] = trial + 1
                df.loc[rowIndex,"Stay_Probability(Colour)"] = stayProbs[0]
                df.loc[rowIndex,"Stay_Probability(Texture)"] = stayProbs[1]
                df.loc[rowIndex,"Stay_Probability(Shape)"] = stayProbs[2]

                stimNodeDimensionSpeedMap = {
                    "Colour": None,
                    "Texture": None,
                    "Shape": None
                }
            
                if trial == 0: #for the very first trial generate 3 random numbers (integers)
                    color_int = random.randint(0,6)
                    texture_int = random.randint(0,6)
                    shape_int = random.randint(0,6)

                    df.loc[rowIndex, "Colour_stim"] = color_int
                    df.loc[rowIndex, "Texture_stim"] = texture_int
                    df.loc[rowIndex, "Shape_stim"] = shape_int

                    currentRow_colour = color_int
                    currentRow_texture = texture_int
                    currentRow_shape = shape_int

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]


                else:
                    color_int = float(f"{random.uniform(0,1):.2f}")
                    texture_int = float(f"{random.uniform(0,1):.2f}")
                    shape_int = float(f"{random.uniform(0,1):.2f}")

                    ##For the Colour dimension
                    
                    if color_int > stayProbs[0]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number is greater than stay probability, we can transition")

                        colour_node = colour_transition_matrix[currentRow_colour] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((colour_node != 0) & (colour_node != stayProbs[0]))
                        adj_node_transprobs = [colour_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {colour_node}, Stay probability : {stayProbs[0]}, Random Number Drawn:{color_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")
                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_colour == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour
                        
                        else:
                            if color_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            elif color_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                        

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number: {color_int} is lower than stay probability {stayProbs[0]}, we stay", "\n")
                        df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                    ##For the texture dimension

                    if texture_int > stayProbs[1]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number is greater than stay probability, we can transition")
                        texture_node = texture_transition_matrix[currentRow_texture] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((texture_node != 0) & (texture_node != stayProbs[1]))
                        adj_node_transprobs = [texture_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {texture_node}, Stay probability : {stayProbs[1]}, Random Number Drawn:{texture_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_texture == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture
                        
                        else:
                            if texture_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")

                            elif texture_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number: {texture_int} is lower than stay probability {stayProbs[1]}, we stay", "\n")
                        df.loc[rowIndex, "Texture_stim"] = currentRow_texture


                    ##For the shape dimension
                    
                    if shape_int > stayProbs[2]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number is greater than stay probability, we can transition")
                        shape_node = shape_transition_matrix[currentRow_shape] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((shape_node != 0) & (shape_node != stayProbs[2]))
                        adj_node_transprobs = [shape_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {shape_node}, Stay probability : {stayProbs[2]}, Random Number Drawn:{shape_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_shape == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape
                        
                        else:
                            if shape_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {shape_node}","\n")

                            elif shape_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number: {shape_int} is lower than stay probability {stayProbs[2]}, we stay", "\n")
                        df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]

                    if trial in choiceTrialFlag :
                        color_int_distractor = random.randint(0,6)
                        texture_int_distractor = random.randint(0,6)
                        shape_int_distractor = random.randint(0,6)

                        #checking if the distractor dimensions are consistent with the preceeding stimulus object
                        #get the stimlus object nodes from preceeding trial, so df.loc[trial - 1, "Colour_stim"]
                        #get also the stimulus object from preceding trial based off speed, so df.loc[trial-1, "Stim_fast or whatever"]
                        #get the neighbour array from the possible neighbours map, if any or all of the distractor stimulus nodes are in the array, it means
                        #they are consistent and therefore the respective dimension gets marked with a 1 and inconsistent ones with a 0
                        relevantCols = {
                            "dimensions" : ["Colour_stim", "Texture_stim", "Shape_stim"],
                            "dimensions_speed": ["Slow_stim", "Medium_stim", "Fast_stim"]
                        }
                        
                        colour_preced = None
                        texture_preced = None
                        shape_preced = None
                        slow_preced = None
                        medium_preced = None
                        fast_preced = None

                        for col in relevantCols:
                            if col == "dimensions":
                                colour_preced = df.loc[rowIndex-1, relevantCols[col][0]]
                                texture_preced = df.loc[rowIndex-1, relevantCols[col][1]]
                                shape_preced = df.loc[rowIndex-1, relevantCols[col][2]]
                            else:
                                slow_preced = df.loc[rowIndex-1, relevantCols[col][0]]
                                medium_preced = df.loc[rowIndex-1, relevantCols[col][1]]
                                fast_preced = df.loc[rowIndex-1, relevantCols[col][2]]

                        nodes_preced = [colour_preced, texture_preced,shape_preced]
                        nodes_speed_preced = [slow_preced,medium_preced,fast_preced]
                        distractorSpeedVals = [
                            df.loc[rowIndex, "Slow_distractor"],
                            df.loc[rowIndex, "Medium_distractor"],
                            df.loc[rowIndex, "Fast_distractor"],
                        ]

                        consistentCols = ["Distractor_consistent_colour",
                                          "Distractor_consistent_texture",
                                          "Distractor_consistent_shape"]
                        
                        consistentSpeedCols = ["Distractor_consistent_slow",
                                               "Distractor_consistent_medium",
                                               "Distractor_consistent_fast"]
                        
                        match_count = 0
                        if color_int_distractor in possibleNeighbours[colour_preced]:
                            match_count += 1
                        if texture_int_distractor in possibleNeighbours[texture_preced]:
                            match_count +=1 
                        if shape_int_distractor in possibleNeighbours[shape_preced]:
                            match_count += 1
                        
                        if match_count ==3:
                            choiceTrialFlag.remove(trial)

                            if trial + 1 < nTrials:
                                choiceTrialFlag.append(trial+1)
                            continue



                        # while (
                        #     #ensure distractor shares no dimension with preceding stimulus
                        #     (color_int_distractor not in possibleNeighbours[colour_preced]) and \
                        #     (shape_int_distractor not in possibleNeighbours [shape_preced]) and \
                        #     (texture_int_distractor not in possibleNeighbours[texture_preced]) 
                            
                        #     # or (
                        #     # #check if the distractor node values are identical to the preceding stimulus object
                        #     # (color_int_distractor == colour_preced) and \
                        #     # (shape_int_distractor == shape_preced) and \
                        #     # (texture_int_distractor == texture_preced)
                        #     # )

                        #     # or
                        #     #ensures not all distractor node values are neighbours of the preceding stimulus object 
                        #     # (
                        #     # (color_int_distractor in possibleNeighbours[colour_preced]) and \
                        #     # (shape_int_distractor in possibleNeighbours [shape_preced]) and \
                        #     # (texture_int_distractor in possibleNeighbours[texture_preced])
                        #     # )
                        #     ):
                            
                        #     ##print("Distractor shares all dimensions")
                        #     color_int_distractor = random.randint(0,6)
                        #     texture_int_distractor = random.randint(0,6)
                        #     shape_int_distractor = random.randint(0,6)      

                       
                        

                        distractorNodeDimensionsMap = {
                            "Colour" : color_int_distractor,
                            "Texture" : texture_int_distractor,
                            "Shape" : shape_int_distractor
                        }
                        
                        df.loc[rowIndex, "Choice Trial Index"] = choiceTrialIndex
                        df.loc[rowIndex, "Colour_distractor"] = color_int_distractor
                        df.loc[rowIndex, "Texture_distractor"] = texture_int_distractor
                        df.loc[rowIndex, "Shape_distractor"] = shape_int_distractor
                        df.loc[rowIndex, "Slow_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Slow"]]
                        df.loc[rowIndex, "Medium_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Medium"]]
                        df.loc[rowIndex, "Fast_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]

                        stimNodeVals = [currentRow_colour,currentRow_texture,currentRow_shape]
                        distractorNodeVals = [color_int_distractor, texture_int_distractor, shape_int_distractor]
                        colsToModify = ["Distractor_shares_colour", "Distractor_shares_texture","Distractor_shares_shape"]

                        stimNodeSpeedVals = [
                            stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]],
                            stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]],
                            stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]
                        ]
                        distractorNodeSpeedVals = [
                            distractorNodeDimensionsMap[dimensionSpeeds["Slow"]],
                            distractorNodeDimensionsMap[dimensionSpeeds["Medium"]],
                            distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]
                        ]
                        speedColsToModify = ["Distractor_shares_slow", "Distractor_shares_medium", "Distractor_shares_fast"]
                    

                        for index, value in enumerate(distractorNodeVals):
                            if value == stimNodeVals[index]:
                                df.loc[rowIndex, colsToModify[index]] = 1
                            else:
                                df.loc[rowIndex,colsToModify[index]] = 0

                        for index, value in enumerate(distractorNodeSpeedVals):
                            if value == stimNodeSpeedVals[index]:
                                df.loc[rowIndex,speedColsToModify[index]] = 1
                            else:
                                df.loc[rowIndex,speedColsToModify[index]] = 0

                     
                        #if distractor dimension values are consistent with the preceding stimulus object == 1, else == 0
                        for index,val in enumerate(nodes_preced):
                            if distractorNodeVals[index] in possibleNeighbours[val]:
                                df.loc[rowIndex, consistentCols[index]] = 1
                            else:
                                df.loc[rowIndex, consistentCols[index]] = 0

                        for index,val in enumerate(nodes_speed_preced):
                            if distractorSpeedVals[index] in possibleNeighbours[val]:
                                df.loc[rowIndex, consistentSpeedCols[index]] = 1
                            else:
                                df.loc[rowIndex, consistentSpeedCols[index]] = 0


                        
                    
                        choiceTrialIndex += 1
                        

                rowIndex += 1

        

        df.iloc[:,0] = participant + 1
        #df.to_csv(f"../Experiment (No Backend)/public/DesignFiles/DesignFile_{participant+1}_session{session+1}.csv", index=False)
        df.to_csv(f"DesignFile_{participant+1}_session{session+1}.csv", index=False)
        
                    

                






C:\Users\Rishi\AppData\Local\Temp\ipykernel_17904\3888414171.py:440: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,0] = participant + 1
C:\Users\Rishi\AppData\Local\Temp\ipykernel_17904\3888414171.py:440: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,0] = participant + 1


In [ ]:
df

,Participant_ID,Block Number,Session Number,Trial Number,Stay_Probability(Colour),Stay_Probability(Texture),Stay_Probability(Shape),Slow_stim,Medium_stim,Fast_stim,...,Distractor_consistent_shape,Distractor_consistent_slow,Distractor_consistent_medium,Distractor_consistent_fast,Distractor_shares_colour,Distractor_shares_texture,Distractor_shares_shape,Distractor_shares_slow,Distractor_shares_medium,Distractor_shares_fast
0,1,1,2,1,0.1,0.3,0.5,2,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2,2,0.1,0.3,0.5,6,2,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,2,3,0.1,0.3,0.5,6,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,2,4,0.1,0.3,0.5,3,2,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,2,5,0.1,0.3,0.5,4,3,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,1,8,2,196,0.1,0.3,0.5,2,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1596,1,8,2,197,0.1,0.3,0.5,3,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1597,1,8,2,198,0.1,0.3,0.5,3,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1598,1,8,2,199,0.1,0.3,0.5,4,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.columns

Index(['Participant_ID', 'Block Number', 'Session Number', 'Trial Number',
       'Stay_Probability(Colour)', 'Stay_Probability(Texture)',
       'Stay_Probability(Shape)', 'Slow_stim', 'Medium_stim', 'Fast_stim',
       'Colour_stim', 'Texture_stim', 'Shape_stim', 'Choice Trial Index',
       'Colour_distractor', 'Texture_distractor', 'Shape_distractor',
       'Slow_distractor', 'Medium_distractor', 'Fast_distractor',
       'Distractor_consistent_colour', 'Distractor_consistent_texture',
       'Distractor_consistent_shape', 'Distractor_consistent_slow',
       'Distractor_consistent_medium', 'Distractor_consistent_fast',
       'Distractor_shares_colour', 'Distractor_shares_texture',
       'Distractor_shares_shape', 'Distractor_shares_slow',
       'Distractor_shares_medium', 'Distractor_shares_fast'],
      dtype='object')

<h3>Creating a json file with the list of the folders in the design file folder</h3>

In [ ]:
import os
import json

In [ ]:
designFileList = os.listdir("../Experiment (No Backend)/public/DesignFiles")
# designFileList.remove("DesignFileList.json")

with open("../Experiment (No Backend)/public/DesignFiles/DesignFileList.json", "w") as jsonFile:
    json.dump(designFileList, jsonFile)

